In [1]:
!nvidia-smi

Sat May 11 23:08:33 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.61                 Driver Version: 551.61         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2060      WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   62C    P0             26W /   80W |    1014MiB /   6144MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
from torch import nn,Tensor
from d2l import torch as d2l

In [4]:
torch.device('cpu'),torch.device('cuda'),torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [5]:
torch.cuda.device_count()

1

In [6]:
def try_gpu(i=0):
  if torch.cuda.device_count() >= i+1:
    return torch.device(f'cuda:{i}')
  return torch.device('cpu')


def try_all_gpus():
  devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
  return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [7]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [10]:
X = torch.ones(2,3,device=try_gpu())

In [12]:
Y = torch.rand(2, 3, device=try_gpu())
Y

tensor([[0.3069, 0.9558, 0.8656],
        [0.5339, 0.4592, 0.9275]], device='cuda:0')

In [13]:
X + Y

tensor([[1.3069, 1.9558, 1.8656],
        [1.5339, 1.4592, 1.9275]], device='cuda:0')

In [14]:
net = nn.Sequential(nn.Linear(3,1))
net = net.to(device=try_gpu())

In [15]:
net(X)

tensor([[-0.7035],
        [-0.7035]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [16]:
net[0].weight.data.device

device(type='cuda', index=0)

In [17]:
import time
startTime1=time.time()
for i in range(100):
    A = torch.ones(500,500)
    B = torch.ones(500,500)
    C = torch.matmul(A,B)
endTime1=time.time()

startTime2=time.time()
for i in range(100):
    A = torch.ones(500,500,device=try_gpu())
    B = torch.ones(500,500,device=try_gpu())
    C = torch.matmul(A,B)
endTime2=time.time()

print('cpu计算总时长:', round((endTime1 - startTime1)*1000, 2),'ms')
print('gpu计算总时长:', round((endTime2 - startTime2)*1000, 2),'ms')


cpu计算总时长: 420.35 ms
gpu计算总时长: 33.62 ms
